In [2]:
import pandas as pd
df_target = pd.read_csv('data/02_input_target.csv')
df_capacity = pd.read_csv('data/02_input_capacity.csv')
df_shipment_cost = pd.read_csv('data/02_03_input_shipmentsCost_example.csv')
df_production_cost = pd.read_csv('data/03_input_productionCost.csv')
df_shipment_cost

,Origin,Destination,Unit Cost
0,Japan,Australia,13.43
1,Japan,Brazil,10.42
2,Japan,Egypt,9.90
3,Japan,Italy,13.01
4,Japan,South Africa,8.92
5,Japan,France,5.60
6,Japan,India,6.75
7,Japan,United Kingdom,10.12
8,Japan,Russia,12.60
9,Australia,Brazil,10.07


In [ ]:
import pandas as pd
from ortools.linear_solver import pywraplp

# Carica i dataset necessari
demand_df = df_target       # Country, Product, Month, Quantity
capacity_df = df_capacity     # Country, Monthly Capacity
shipment_cost_df = df_shipment_cost # Origin, Destination, Product, Cost_per_Unit
production_cost_df = df_production_cost # Country, Product, Cost_per_Unit

# Prepara dizionari
demand_df['Key'] = list(zip(demand_df['Country'], demand_df['Product'], demand_df['Month']))
demand = dict(zip(demand_df['Key'], demand_df['Quantity']))

capacity = dict(zip(capacity_df['Country'], capacity_df['Monthly Capacity']))

# Creiamo un dizionario per i costi di trasporto
transport_costs = {}
for _, row in shipment_cost_df.iterrows():
    key = (row['Origin'], row['Destination'])
    transport_costs[key] = row['Unit Cost']

# Creiamo un dizionario per i costi di produzione
production_costs = {}
for _, row in production_cost_df.iterrows():
    key = (row['Country'], row['Product'])
    production_costs[key] = row['Unit Cost']

# Impianti e mesi
plants = list(capacity.keys())
months = sorted(demand_df['Month'].unique())
demand_keys = list(demand.keys())

# Inizializza il solver OR-Tools
solver = pywraplp.Solver.CreateSolver('SAT')

# Variabili decisionali: quanto produce ogni impianto per soddisfare domanda (i,j,p,m)
x = {}
for i in plants:
    for (j, p, m) in demand_keys:
        x[i, j, p, m] = solver.NumVar(0, solver.infinity(), f'x_{i}_{j}_{p}_{m}')

# Variabile per il massimo carico (da minimizzare)
max_load = solver.NumVar(0, 1, 'max_load')

# 1. Vincolo di domanda
for (j, p, m) in demand_keys:
    solver.Add(solver.Sum(x[i, j, p, m] for i in plants) == demand[(j, p, m)])

# 2. Vincoli di capacità impianto
for i in plants:
    for m in months:
        monthly_prod = solver.Sum(x[i, j, p, m] for (j, p, m2) in demand_keys if m2 == m)
        solver.Add(monthly_prod <= capacity[i])

# 3. Vincoli per il carico massimo
for i in plants:
    for m in months:
        monthly_prod = solver.Sum(x[i, j, p, m] for (j, p, m2) in demand_keys if m2 == m)
        solver.Add(monthly_prod <= max_load * capacity[i])

# 4. Funzione obiettivo: Minimizzare i costi di produzione e trasporto
total_cost = solver.Sum(
    x[i, j, p, m] * transport_costs.get((i, j, p), 0) # Costo di trasporto
    for i in plants for (j, p, m) in demand_keys
)

# Aggiungi i costi di produzione alla funzione obiettivo
total_cost += solver.Sum(
    x[i, j, p, m] * production_costs.get((i, p), 0)  # Costo di produzione
    for i in plants for (j, p, m) in demand_keys
)

# Aggiungi l'obiettivo (costo totale da minimizzare)
# solver.Minimize(total_cost * 1e-7 + max_load)

# Risoluzione
status = solver.Solve()

# Output della soluzione
if status == pywraplp.Solver.OPTIMAL:
    print(f"Status: Ottimale")
    print(f"Carico massimo usato: {max_load.solution_value():.2%}")
    print("Costi totali:", solver.Objective().Value())
    
    # Crea due liste separate per il piano di produzione e quello di spedizione
    production_data = []
    shipment_data = []

    for (i, j, p, m), var in x.items():
        if var.solution_value() > 0:
            # Piano di produzione: quantità prodotta per ogni impianto
            production_data.append({
                'Country': i,
                'Product': p,
                'Month': m,
                'Quantity': int(var.solution_value())
            })

            # Piano di spedizione: quantità spedita tra impianto e mercato
            shipment_data.append({
                'Origin': i,
                'Destination': j,
                'Product': p,
                'Month': m,
                'Quantity': int(var.solution_value())
            })
    
    # Crea DataFrame per i piani di produzione e spedizione
    production_plan = pd.DataFrame(production_data)
    shipment_plan = pd.DataFrame(shipment_data)

    # Stampa i risultati
    print("Production Plan:")
    print(production_plan)

    print("\nShipment Plan:")
    print(shipment_plan)

else:
    print('Nessuna soluzione ottimale trovata.')


Status: Ottimale
Carico massimo usato: 100.00%
Costi totali: 8.533171729000014
Production Plan:
             Country                           Product    Month  Quantity
0          Australia              BreatheEasy Inhalant  Jan2004      1160
1          Australia              BreatheEasy Inhalant  Feb2004      1187
2          Australia              BreatheEasy Inhalant  Mar2004      1056
3          Australia              BreatheEasy Inhalant  Apr2004      1134
4          Australia              BreatheEasy Inhalant  May2004      1134
...              ...                               ...      ...       ...
7397  United Kingdom  BrightBreeze Pet Odor Eliminator  Aug2004       821
7398  United Kingdom  BrightBreeze Pet Odor Eliminator  Sep2004       684
7399  United Kingdom  BrightBreeze Pet Odor Eliminator  Oct2004       623
7400  United Kingdom  BrightBreeze Pet Odor Eliminator  Nov2004       547
7401  United Kingdom  BrightBreeze Pet Odor Eliminator  Dec2004       835

[7402 rows x 4 

In [35]:
# Prepara dizionari
demand_df['Key'] = list(zip(demand_df['Country'], demand_df['Product'], demand_df['Month']))
demand = dict(zip(demand_df['Key'], demand_df['Quantity']))

capacity = dict(zip(capacity_df['Country'], capacity_df['Monthly Capacity']))

# Creiamo un dizionario per i costi di trasporto (basato su Origin e Destination)
transport_costs = {}
for _, row in shipment_cost_df.iterrows():
    key = (row['Origin'], row['Destination'])
    transport_costs[key] = row['Unit Cost']

# Creiamo un dizionario per i costi di produzione
production_costs = {}
for _, row in production_cost_df.iterrows():
    key = (row['Country'], row['Product'])
    production_costs[key] = row['Unit Cost']

# Impianti e mesi
plants = list(capacity.keys())
months = sorted(demand_df['Month'].unique())
demand_keys = list(demand.keys())

# Inizializza il solver OR-Tools
solver = pywraplp.Solver.CreateSolver('SCIP')

# Variabili decisionali: quanto produce ogni impianto per soddisfare domanda (i,j,p,m)
x = {}
for i in plants:
    for (j, p, m) in demand_keys:
        x[i, j, p, m] = solver.NumVar(0, solver.infinity(), f'x_{i}_{j}_{p}_{m}')

# Variabile per il massimo carico (da minimizzare)
max_load = solver.NumVar(0, 1, 'max_load')

# 1. Vincolo di domanda
for (j, p, m) in demand_keys:
    solver.Add(solver.Sum(x[i, j, p, m] for i in plants) == demand[(j, p, m)])

# 2. Vincoli di capacità impianto
for i in plants:
    for m in months:
        monthly_prod = solver.Sum(x[i, j, p, m] for (j, p, m2) in demand_keys if m2 == m)
        solver.Add(monthly_prod <= capacity[i])

# 3. Vincoli per il carico massimo
for i in plants:
    for m in months:
        monthly_prod = solver.Sum(x[i, j, p, m] for (j, p, m2) in demand_keys if m2 == m)
        solver.Add(monthly_prod <= max_load * capacity[i])

# 4. Funzione obiettivo: Minimizzare i costi di produzione e trasporto
total_cost = solver.Sum(
    x[i, j, p, m] * transport_costs.get((i, j), 0)  # Costo di trasporto (senza dipendenza dal prodotto)
    for i in plants for (j, p, m) in demand_keys
)

# Aggiungi i costi di produzione alla funzione obiettivo
total_cost += solver.Sum(
    x[i, j, p, m] * production_costs.get((i, p), 0)  # Costo di produzione
    for i in plants for (j, p, m) in demand_keys
)

# Aggiungi l'obiettivo (costo totale da minimizzare)
solver.Minimize(total_cost + max_load)

# Risoluzione
status = solver.Solve()

# Output della soluzione
if status == pywraplp.Solver.OPTIMAL:
    print(f"Status: Ottimale")
    print(f"Carico massimo usato: {max_load.solution_value():.2%}")
    print("Costi totali:", solver.Objective().Value())
    
    # Crea due liste separate per il piano di produzione e quello di spedizione
    production_data = []
    shipment_data = []

    for (i, j, p, m), var in x.items():
        if var.solution_value() > 0:
            # Piano di produzione: quantità prodotta per ogni impianto
            production_data.append({
                'Country': i,
                'Product': p,
                'Month': m,
                'Quantity': int(var.solution_value())
            })

            # Piano di spedizione: quantità spedita tra impianto e mercato
            shipment_data.append({
                'Origin': i,
                'Destination': j,
                'Product': p,
                'Month': m,
                'Quantity': int(var.solution_value())
            })
    
    # Crea DataFrame per i piani di produzione e spedizione
    production_plan = pd.DataFrame(production_data)
    shipment_plan = pd.DataFrame(shipment_data)

    # Stampa i risultati
    print("Production Plan:")
    print(production_plan)

    print("\nShipment Plan:")
    print(shipment_plan)

else:
    print('Nessuna soluzione ottimale trovata.')

Status: Ottimale
Carico massimo usato: 100.00%
Costi totali: 90970405.49
Production Plan:
             Country                   Product    Month  Quantity
0          Australia      BreatheEasy Inhalant  Jan2004      1160
1          Australia      BreatheEasy Inhalant  Feb2004      1187
2          Australia      BreatheEasy Inhalant  Mar2004      1056
3          Australia      BreatheEasy Inhalant  Apr2004      1134
4          Australia      BreatheEasy Inhalant  May2004      1134
...              ...                       ...      ...       ...
7975  United Kingdom  HydratingHoney Pet Wipes  Aug2004      1169
7976  United Kingdom  HydratingHoney Pet Wipes  Sep2004      1189
7977  United Kingdom  HydratingHoney Pet Wipes  Oct2004      1128
7978  United Kingdom  HydratingHoney Pet Wipes  Nov2004      1128
7979  United Kingdom  HydratingHoney Pet Wipes  Dec2004      1215

[7980 rows x 4 columns]

Shipment Plan:
              Origin     Destination                   Product    Month  \
0 

In [36]:
production_plan.to_csv('outputs/03_output_productionPlan_1239.csv', index=False)
shipment_plan.to_csv('outputs/03_output_shipments_1239.csv', index=False)